In [1]:
# !pip install mecab-python3
#These wheels include a copy of the MeCab library, but not a dictionary. 
#In order to use MeCab you'll need to install a dictionary. unidic-lite is a good one to start with:
# !pip install unidic-lite

In [2]:
# normalization tool
# !pip install neologdn

In [3]:
# !pip install openpyxl

In [4]:
# # To be able to see Japanese!
# !pip install japanize_matplotlib

everything was deleted what a christmas miracle....
lets start again ...


## Librarys

In [5]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

import os
import pandas as pd

# Preprocessing
import MeCab
import neologdn
import collections
from nltk import FreqDist
from nltk.corpus import stopwords

# Visualization
import matplotlib.pyplot as plt
import japanize_matplotlib
#import seaborn as sns # REMINDER: make sure to remove if not using!

In [6]:
file_name = 'Combined_85K_150.xlsx'
df = pd.read_excel(r'/home/samuelhenderson/code/simply-japanese/data/2_RawData/' + file_name)
df

,#日本語(原文),#やさしい日本語,#英語(原文)
0,彼女は通りを横切った。,彼女は通りを横に通っていった。,she came across the street .
1,私が知るかぎり彼女は大変よい人だ。,私が知る限り彼女は大変よい人だ。,"as far as i know , she is a very good person ."
2,私のクラスの少女たちはみんな親切だ。,私のクラスの少女たちはみんな親切だ。,all the girls in my class are kind .
3,彼は試験に合格できなかった。,彼は試験に合格できなかった。,he couldn 't pass the examination .
4,彼女はあなたにあえて喜ぶでしょうね。,彼女はあなたに会うことができて喜ぶでしょうね。,"she 'll be glad to see you , won 't she ?"
...,...,...,...
145,いとこは私を町中案内してくれた。,いとこは私に町の中を案内してくれた。,My cousin took me around the town.
146,木の葉が赤や黄色になり始めた。,森が赤い色や黄色になりはじめた。,The leaves began to turn red and yellow.
147,今頃ロンドンは寒いに違いない。,今の季節ロンドンは寒いに違いない。,London must be cold now.
148,このスープは何か足した方がいいんじゃないかしら。,このスープは何か加えたほうがいいのではないですか。,This soup needs something to pick it up.


In [7]:
df.drop(columns=['#英語(原文)'], inplace=True)
df.head()

,#日本語(原文),#やさしい日本語
0,彼女は通りを横切った。,彼女は通りを横に通っていった。
1,私が知るかぎり彼女は大変よい人だ。,私が知る限り彼女は大変よい人だ。
2,私のクラスの少女たちはみんな親切だ。,私のクラスの少女たちはみんな親切だ。
3,彼は試験に合格できなかった。,彼は試験に合格できなかった。
4,彼女はあなたにあえて喜ぶでしょうね。,彼女はあなたに会うことができて喜ぶでしょうね。


In [8]:
df.rename(columns={"#日本語(原文)": "original", "#やさしい日本語": "simplified"}, inplace=True)
df.head()

,original,simplified
0,彼女は通りを横切った。,彼女は通りを横に通っていった。
1,私が知るかぎり彼女は大変よい人だ。,私が知る限り彼女は大変よい人だ。
2,私のクラスの少女たちはみんな親切だ。,私のクラスの少女たちはみんな親切だ。
3,彼は試験に合格できなかった。,彼は試験に合格できなかった。
4,彼女はあなたにあえて喜ぶでしょうね。,彼女はあなたに会うことができて喜ぶでしょうね。


In [9]:
# cosine similarity 
# X= df['original']
# y= df['simplified']

# cosine_similarity(X, y)

In [10]:
#testing out stuff from github repository : 
# https://github.com/ZHAOTING/Japanese-sentence-similarity-scorer

In [11]:
!python -m unidic download


/home/samuelhenderson/.pyenv/versions/simply-japanese/bin/python: No module named unidic


In [12]:
# #fast text
# import fasttext
# import fasttext.util

In [13]:
fasttext.util.download_model('jp', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

NameError: name 'fasttext' is not defined

## SumEval
https://github.com/chakki-works/sumeval#welcome-contribution-tada

In [ ]:
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# !python -m spacy download en_core_web_sm

In [ ]:
# pip install sumeval==0.1.5

In [ ]:
# pip uninstall sumeval

In [ ]:
# pip install janome

In [ ]:
import sacrebleu

In [ ]:
from sacrebleu import corpus_bleu, DEFAULT_TOKENIZER

In [ ]:
type(df.iloc[0][0])

In [ ]:
from sumeval.metrics.bleu import BLEUCalculator
# https://github.com/chakki-works/sumeval
from janome.tokenizer import Tokenizer


# bleu = BLEUCalculator()
# score = bleu.bleu("I am waiting on the beach",
#                   "He is walking on the beach")

bleu_ja = BLEUCalculator(lang="ja")
score_ja = bleu_ja.bleu(df.iloc[0][0], df.iloc[0][1])
tokenizer_ja()

In [18]:
from sumeval.metrics.bleu import BLEUCalculator


bleu = BLEUCalculator()
score = bleu.bleu("I am waiting on the beach",
                  "He is walking on the beach")

# bleu_ja = BLEUCalculator(lang="ja")
# score_ja = bleu_ja.bleu("私はビーチで待ってる", "彼がベンチで待ってる")

In [15]:
from sacrebleu import ja_mecab

ImportError: cannot import name 'ja_mecab' from 'sacrebleu' (/home/samuelhenderson/.pyenv/versions/3.8.12/envs/simply-japanese/lib/python3.8/site-packages/sacrebleu/__init__.py)

In [ ]:
bleu = BLEUCalculator()

In [ ]:
df.iloc[0][0]

In [17]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu_ja = BLEUCalculator(lang="ja")
score_ja = bleu_ja.bleu(df.iloc[0][0], df.iloc[0][1])


TypeError: tokenizer_ja() missing 1 required positional argument: 'text'

In [ ]:
import sacrebleu

In [ ]:
sacrebleu.info()

In [16]:
from sacrebleu.metrics import BLEU, CHRF, TER

refs = [ # First set of references
          ['The dog bit the man.', 'It was not unexpected.', 'The man bit him first.'],
          # Second set of references
          ['The dog had bit the man.', 'No one was surprised.', 'The man had bitten the dog.'],
        ]
sys = ['The dog bit the man.', "It wasn't surprising.", 'The man had just bitten him.']

bleu = BLEU()

bleu.corpus_score(sys, refs)
# BLEU = 48.53 82.4/50.0/45.5/37.5 (BP = 0.943 ratio = 0.944 hyp_len = 17 ref_len = 18)

bleu.get_signature()
# nrefs:2|case:mixed|eff:no|tok:13a|smooth:exp|version:2.0.0

chrf = CHRF()

chrf.corpus_score(sys, refs)
# chrF2 = 59.73

TypeError: __init__() missing 1 required positional argument: 'args'

In [ ]:
!pip list